<a href="https://colab.research.google.com/github/naresh-FD/react-testgen-coverage/blob/main/react_testgen_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q "torch>=2.1" transformers datasets peft accelerate bitsandbytes trl



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 15.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!ls -lh /content/drive/MyDrive

total 406K
drwx------ 2 root root 4.0K Feb 10 05:49 'Colab Notebooks'
drwx------ 2 root root 4.0K Jan 31 06:16  Documents
drwx------ 2 root root 4.0K Feb  1 05:36 'Exchange events attachments'
-rw------- 1 root root 390K Feb 10 05:46  react_testgen_train.ipynb
drwx------ 2 root root 4.0K Jan 31 06:16  Social


In [4]:
cp /content/drive/MyDrive/training.jsonl /content/training.jsonl

cp: cannot stat '/content/drive/MyDrive/training.jsonl': No such file or directory


In [5]:
!ls -lh /content
!head -n 2 /content/training.jsonl


total 8.0K
drwx------ 5 root root 4.0K Feb 10 05:51 drive
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data
head: cannot open '/content/training.jsonl' for reading: No such file or directory


In [6]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="/content/training.jsonl"
)["train"]

print("Total examples:", len(dataset))
print(dataset[0]["messages"][1]["content"][:300])

FileNotFoundError: Unable to find '/content/training.jsonl'

In [ ]:
print(dataset.column_names)
print(dataset[0]["messages"][0])
print(dataset[0]["messages"][1])


In [ ]:
def to_text(ex):
    # assumes messages = [{role, content}, {role, content}, ...]
    msgs = ex["messages"]
    # minimal format: user prompt -> assistant answer
    # if you have system+user+assistant, this still works well
    user_parts = [m["content"] for m in msgs if m["role"] in ["user", "system"]]
    assistant_parts = [m["content"] for m in msgs if m["role"] == "assistant"]

    prompt = "\n\n".join(user_parts).strip()
    completion = "\n\n".join(assistant_parts).strip()

    return {"text": f"{prompt}\n\n### RESPONSE:\n{completion}"}

dataset_text = dataset.map(to_text)
print(dataset_text[0]["text"][:600])


In [ ]:
!pip install -q "torch>=2.1" transformers datasets peft accelerate bitsandbytes trl


In [ ]:
BASE_MODEL = "Qwen/Qwen2.5-Coder-1.5B-Instruct"


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
)

lora = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
)

model = get_peft_model(model, lora)
model.print_trainable_parameters()


In [ ]:
def tokenize(ex):
    out = tok(
        ex["text"],
        truncation=True,
        max_length=2048,
        padding=False,
    )
    out["labels"] = out["input_ids"].copy()
    return out

train_ds = dataset_text.map(tokenize, remove_columns=dataset_text.column_names)
print(train_ds[0].keys())


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/content/out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=5,          # small dataset => more epochs ok
    logging_steps=5,
    save_steps=50,
    bf16=True, # Changed from fp16=True
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    args=args
)

trainer.train()

In [ ]:
model.save_pretrained("/content/react-testgen-lora")
tok.save_pretrained("/content/react-testgen-lora")
!ls -lh /content/react-testgen-lora


In [ ]:
import torch

def generate(prompt: str):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=600,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
    return tok.decode(out[0], skip_special_tokens=True)

test_prompt = dataset_text[0]["text"].split("### RESPONSE:")[0].strip() + "\n\n### RESPONSE:\n"
print(generate(test_prompt))


In [ ]:
def to_text(ex):
    msgs = ex["messages"]
    # Combine system+user into prompt, assistant into completion
    prompt = "\n\n".join([m["content"] for m in msgs if m["role"] in ["system","user"]]).strip()
    completion = "\n\n".join([m["content"] for m in msgs if m["role"] == "assistant"]).strip()
    return {"text": f"{prompt}\n\n### RESPONSE:\n{completion}"}

dataset_text = dataset.map(to_text)
print(dataset_text[0]["text"][:600])


In [ ]:
from datasets import load_dataset

# Load a sample dataset (e.g., a small subset of alpaca)
# Alpaca dataset structure: 'instruction', 'input', 'output'
dataset = load_dataset("tatsu-lab/alpaca", split="train[:50]") # Load first 50 examples

print("Total examples:", len(dataset))
# Print an example of the new dataset's content
print(dataset[0]["instruction"][:300])

In [ ]:
def to_text(ex):
    prompt_parts = []
    if ex["instruction"]:
        prompt_parts.append(ex["instruction"])
    if ex["input"]:
        prompt_parts.append(f"Input: {ex['input']}")

    prompt = "\n\n".join(prompt_parts).strip()
    completion = ex["output"].strip()

    return {"text": f"{prompt}\n\n### RESPONSE:\n{completion}"}

dataset_text = dataset.map(to_text)
print(dataset_text[0]["text"][:600])

In [ ]:
!pip install -q "torch>=2.1" transformers datasets peft accelerate bitsandbytes trl


In [ ]:
def tokenize(ex):
    out = tok(
        ex["text"],
        truncation=True,
        max_length=2048,
        padding=False,
    )
    out["labels"] = out["input_ids"].copy()
    return out

train_ds = dataset_text.map(tokenize, remove_columns=dataset_text.column_names)
print(train_ds[0].keys())

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/content/out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=5,          # small dataset => more epochs ok
    logging_steps=5,
    save_steps=50,
    bf16=True, # Changed from fp16=True
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    args=args
)

trainer.train()

In [ ]:
model.save_pretrained("/content/react-testgen-lora")
tok.save_pretrained("/content/react-testgen-lora")
!ls -lh /content/react-testgen-lora

In [ ]:
import torch

def generate(prompt: str):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=600,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
    return tok.decode(out[0], skip_special_tokens=True)

test_prompt = dataset_text[0]["text"].split("### RESPONSE:")[0].strip() + "\n\n### RESPONSE:\n"
print(generate(test_prompt))

In [ ]:
BASE_MODEL = "Qwen/Qwen2.5-Coder-1.5B-Instruct"


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb,
    device_map="auto",
)

lora = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
)

model = get_peft_model(model, lora)
model.print_trainable_parameters()


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/content/out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=5,
    logging_steps=5,
    save_steps=50,
    fp16=True,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    args=args,
)

trainer.train()


In [ ]:
from datasets import load_dataset

# Make sure you've uploaded training.jsonl to Colab first
dataset = load_dataset("json", data_files="/content/training.jsonl", split="train")

print(f"✅ Loaded {len(dataset)} training examples")
print(f"Sample keys: {dataset[0].keys()}")
print(f"Messages in first example: {len(dataset[0]['messages'])}")


In [ ]:
def format_chat(example):
    """Convert messages array to ChatML format string"""
    text = ""
    for msg in example["messages"]:
        if msg["role"] == "system":
            text += f"<|im_start|>system\n{msg['content']}<|im_end|>\n"
        elif msg["role"] == "user":
            text += f"<|im_start|>user\n{msg['content']}<|im_end|>\n"
        elif msg["role"] == "assistant":
            text += f"<|im_start|>assistant\n{msg['content']}<|im_end|>\n"
    return {"text": text}

formatted_dataset = dataset.map(format_chat)
print(f"✅ Formatted {len(formatted_dataset)} examples")
print(f"Sample text length: {len(formatted_dataset[0]['text'])} chars")

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/output",
    num_train_epochs=4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=5,
    save_steps=50,
    save_total_limit=2,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    max_grad_norm=0.3,
    weight_decay=0.01,
)

print("✅ Training args configured")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Learning rate: {training_args.learning_rate}")

In [ ]:
from trl import SFTTrainer, SFTConfig

# Move all args into SFTConfig (replaces both TrainingArguments and SFTTrainer params)
sft_config = SFTConfig(
    output_dir="/content/output",
    num_train_epochs=4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=5,
    save_steps=50,
    save_total_limit=2,
    bf16=True, # Changed from fp16=True
    optim="paged_adamw_8bit",
    report_to="none",
    max_grad_norm=0.3,
    weight_decay=0.01,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    args=sft_config
)

print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

In [ ]:
 # Save LoRA adapter
trainer.save_model("/content/output/final")
tok.save_pretrained("/content/output/final")

# Merge LoRA weights back into base model
from peft import AutoPeftModelForCausalLM

merged_model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/output/final",
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)
merged_model = merged_model.merge_and_unload()
merged_model.save_pretrained("/content/output/merged", safe_serialization=True)
tok.save_pretrained("/content/output/merged")

print("✅ Model merged and saved!")

In [ ]:
# Install llama.cpp converter
!pip install -q llama-cpp-python

# Clone llama.cpp for conversion script
!git clone --depth 1 https://github.com/ggerganov/llama.cpp /content/llama_cpp

# Install requirements
!pip install -q -r /content/llama_cpp/requirements.txt

# Convert to GGUF
!python /content/llama_cpp/convert_hf_to_gguf.py \
    /content/output/merged \
    --outfile /content/react-testgen.gguf \
    --outtype q4_K_M

import os
size_mb = os.path.getsize("/content/react-testgen.gguf") / (1024*1024)
print(f"✅ GGUF file created: {size_mb:.0f} MB")
print("📥 Download: Click Files panel → right-click react-testgen.gguf → Download")
